In [1]:
import numpy as np

In [2]:
def normdifference_estimation(A, V, u0, v0, max_iter = 100000, tol = 10**-10, solution = None):
    # Initialization
    u = u0
    v = v0

    if np.dot(u.flatten(), A(v).flatten()) - np.dot(V(u).flatten(), v.flatten()) < 0:
        u = -u

    objective = 0

    Vu = V(u)
    Av = A(v)
    i = 0
    while True:
        # sample random vectors
        y = np.random.randn(*v.shape)
        y -=  np.dot(y.flatten(),v.flatten())*v
        x = y/np.linalg.norm(y)

        z = np.random.randn(*u.shape)
        z -= np.dot(z.flatten(),u.flatten())*u
        w = z/np.linalg.norm(z)

        # Calculate a_k and b_k
        Ax = A(x)
        Vw = V(w)

        a = np.dot(w.flatten(), Av.flatten()) - np.dot(Vw.flatten(), v.flatten()) + np.dot(u.flatten(), Ax.flatten())  - np.dot(Vu.flatten(), x.flatten())
        b = 2*(np.dot(w.flatten(),Ax.flatten()) - np.dot(Vw.flatten(), x.flatten()) - np.dot(u.flatten(), Av.flatten()) + np.dot(Vu.flatten(), v.flatten()))

        if a != 0:
            tau = np.sign(a)*(b/(2*np.abs(a)) + np.sqrt(b**2/(4*a**2)+1))
            u += tau*w
            u /= np.linalg.norm(u)
            v += tau*x
            v /= np.linalg.norm(v)
        else:
            if b > 0:
                tau = np.inf
                u = w
                v = x
            else:
                tau = 0

        Vu = V(u)
        Av = A(v)

        i += 1

        # Estimate
        objective = np.dot(u.flatten(), Av.flatten()) - np.dot(Vu.flatten(), v.flatten())

        # Break criteria
        count = 0 if np.abs(a) > tol else count + 1
        if count == 100:
            break
        if solution is not None:
            if np.abs(objective - solution) < tol:
                break
        if i > max_iter:
            break
    

    return objective
        

In [ ]:
import astra

image = np.ones((400,400))

# Create a basic 128x128 square volume geometry
vol_geom = astra.create_vol_geom(image.shape[0], image.shape[1])
angles = 40
detectors = 400

# Create a parallel beam geometry with 40 angles between 0 and pi, and
# 128 detector pixels of width 1.0
proj_geom = astra.create_proj_geom('parallel', 1.0, detectors, np.linspace(0,np.pi,angles,False))


# Create a sinogram
proj_id1 = astra.create_projector('linear',proj_geom,vol_geom)
proj_id2 = astra.create_projector('line',proj_geom,vol_geom)
proj_id3 = astra.create_projector('strip',proj_geom,vol_geom)

# Forward Projection
def Forward(x, proj_id):
    _, sinogram = astra.create_sino(x, proj_id)
    return sinogram

# Backprojection
def Backprojection(sinogram, proj_id):
    [_, volume] = astra.create_backprojection(sinogram, proj_id);
    return volume


for i in range(3):
    exec('proj_R = proj_id%d' % (i+1))

    R = lambda x: Forward(x, proj_R)
    RT = lambda y: Backprojection(y, proj_R)

    sinogram = R(image)

    adjoint_estimate = normdifference_estimation(R, RT, np.ones(sinogram.shape), np.ones(image.shape), max_iter = 1e3, tol = 10**-10)

    print('Adjoint Mismatch for proj_id%d: %f' % (i+1, adjoint_estimate))
    